In [34]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from pathlib import Path

In [35]:
# Create a list of paths of images for training, validation and testing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dir = Path('../input/100-bird-species/birds/train')
val_dir = Path('../input/100-bird-species/birds/valid')
test_dir = Path('../input/100-bird-species/birds/test')
train_data = ImageFolder(train_dir, transform = transform)
test_data = ImageFolder(test_dir, transform = transform)
val_data = ImageFolder(val_dir, transform = transform)

In [36]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size = 32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle=True)

In [37]:
classes = list(train_data.class_to_idx.keys())

In [38]:
# neural network architecture
class NN(nn.Module):
      def __init__(self):
          super(NN, self).__init__()
          self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=2)
          self.batch_norm1 = nn.BatchNorm2d(64)  
          self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
          self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2)
          self.batch_norm2 = nn.BatchNorm2d(128)
          self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
          self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1)
          self.batch_norm3 = nn.BatchNorm2d(256)  
          self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)
          self.drop_out1 = nn.Dropout(0.5)
          self.fc1 = nn.Linear(in_features=4*4*256, out_features=2048)
          self.drop_out2 = nn.Dropout(0.5)
          self.fc2 = nn.Linear(in_features=2048, out_features=2048)
          self.drop_out3 = nn.Dropout(0.5)
          self.fc3 = nn.Linear(in_features=2048, out_features=275)
      def forward(self, x): 
          output = self.pool1(F.relu(self.batch_norm1(self.conv1(x))))
          output = self.pool1(F.relu(self.batch_norm2(self.conv2(output))))
          output = self.pool1(F.relu(self.batch_norm3(self.conv3(output))))
          output = output.flatten(1)
          output = self.drop_out1(output)
          output = F.relu(self.fc1(output))
          output = self.drop_out2(output)
          output = F.relu(self.fc2(output))
          output = self.drop_out3(output)
          output = F.relu(self.fc3(output))
          return output
 
neu_net = NN()
if torch.cuda.is_available():
   neu_net.cuda()
neu_net

NN(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2))
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (batch_norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop_out1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (drop_out2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (drop_out3): Dropout(p=0.5, inplace=Fals

In [43]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(neu_net.parameters(), lr=0.01, momentum=0.9)

In [44]:
# calculate training and validation accuracy
for epoch in range(1,11):
    # create a loop to enumerate over batches 
    train_loss = 0
    val_loss = 0
    correct_predictions_t = 0
    total_predictions_t = 0
    correct_predictions_v = 0
    total_predictions_v = 0
    for inputs, labels in train_loader:
        if torch.cuda.is_available():
           inputs, labels = inputs.cuda(), labels.cuda()   
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward propagation 
        outputs = neu_net(inputs)
        # calculate loss 
        loss_func = loss_function(outputs, labels)
        train_loss += loss_func.item()
        prediction = outputs.argmax(1)  # calculate the index that has maximum value 
        correct_predictions_t += prediction.eq(labels.data).sum().item()
        total_predictions_t += labels.size(0)
        # use the backward call to calculate gradients of loss function wrt the parameters
        loss_func.backward()
        # update the values of parameters
        optimizer.step() 
    for inputs, labels in val_loader:
        if torch.cuda.is_available():
           inputs, labels = inputs.cuda(), labels.cuda()   
        outputs = neu_net(inputs)
        loss_func = loss_function(outputs,labels)
        val_loss += loss_func.item()
        prediction = outputs.argmax(1)  # calculate the index that has maximum value 
        correct_predictions_v += prediction.eq(labels.data).sum().item()
        total_predictions_v += labels.size(0)
    print('Epoch: %d  Training Loss: %.5f  Training Accuracy: %.2f %% Validation Loss: %.5f Validation Accuracy:%.2f %%' % (epoch, train_loss / (len(train_loader)), 100*(correct_predictions_t/total_predictions_t), val_loss / (len(val_loader)), 100*(correct_predictions_v/total_predictions_v)))

Epoch: 1  Training Loss: 0.90388  Training Accuracy: 75.15 % Validation Loss: 1.07462 Validation Accuracy:73.16 %
Epoch: 2  Training Loss: 0.84563  Training Accuracy: 76.85 % Validation Loss: 1.06170 Validation Accuracy:73.60 %
Epoch: 3  Training Loss: 0.81018  Training Accuracy: 77.72 % Validation Loss: 0.97563 Validation Accuracy:75.56 %
Epoch: 4  Training Loss: 0.75868  Training Accuracy: 79.11 % Validation Loss: 1.12146 Validation Accuracy:72.00 %
Epoch: 5  Training Loss: 0.71794  Training Accuracy: 79.96 % Validation Loss: 1.04641 Validation Accuracy:73.09 %
Epoch: 6  Training Loss: 0.68926  Training Accuracy: 80.75 % Validation Loss: 1.03026 Validation Accuracy:75.20 %
Epoch: 7  Training Loss: 0.65088  Training Accuracy: 81.75 % Validation Loss: 1.07713 Validation Accuracy:72.95 %
Epoch: 8  Training Loss: 0.62217  Training Accuracy: 82.50 % Validation Loss: 1.05445 Validation Accuracy:74.47 %
Epoch: 9  Training Loss: 0.61228  Training Accuracy: 82.81 % Validation Loss: 1.08245 Va

In [45]:
# accuracy on test images
correct_predictions = 0
total_predictions = 0
for inputs, labels in test_loader:
    if torch.cuda.is_available():
           inputs, labels = inputs.cuda(), labels.cuda() 
           outputs = neu_net(inputs)
           prediction = outputs.argmax(1)
           correct_predictions += prediction.eq(labels.data).sum().item()
           total_predictions += labels.size(0)
print('Test_Accuracy: %.2f %%' % (100 * (correct_predictions/total_predictions)))

Test_Accuracy: 77.38 %
